In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
animeSet = pd.read_csv('/kaggle/input/anime-recommendations-database/anime.csv', usecols=[0, 1, 2], index_col='anime_id')
animeSet

# We only need Anime Table to lookup the name and genre of the recommended anime Id's 
**We don't need it for recommendation process**

Let's create a method where we pass anime_id and get name and title.

In [ ]:
def getAnimeDetails(anime_id):
    name = animeSet.at[anime_id, 'name']
    genre = animeSet.at[anime_id, 'genre']
    return anime_id, name, genre

In [ ]:
getAnimeDetails(32281)

# Let's Read ratings data and clean that


In [ ]:
ratings = pd.read_csv('/kaggle/input/anime-recommendations-database/rating.csv')
ratings

In [ ]:
print('users are', len(ratings['user_id'].value_counts()))
print('animes are', len(ratings['anime_id'].value_counts()))
print('so the matrix is ', len(ratings['user_id'].value_counts()), 'X', len(ratings['anime_id'].value_counts()))

# It's better to drop users who have seen fewer than 5 Movies
# Drop movies watched by fewwer than 5 users
**This will reduce the size of the sparse matrix**

In [ ]:
animeIdSeries = ratings['anime_id'].value_counts()
userIdSeries = ratings['user_id'].value_counts()
# Remove inactive animies
ratings = ratings[ratings['anime_id'].isin(animeIdSeries[animeIdSeries > 10].index)]
# Remove inactive Users
ratings = ratings[ratings['user_id'].isin(userIdSeries[userIdSeries > 10].index)]
# Remove animies those are not in anime dataset
ratings = ratings[ratings['anime_id'].isin(animeSet.index)]

print('final users are', len(ratings['user_id'].value_counts()))
print('final animes are', len(ratings['anime_id'].value_counts()))
print('so the matrix is ', len(ratings['user_id'].value_counts()), 'X', len(ratings['anime_id'].value_counts()))

# Now we have the good and less sparse table
# Now we can go ahead with creating the pivot table

In [ ]:
# Replace -1 with NaN because -1 will be considered for distance
def replacewithNAN(value):
    if value == -1:
        return np.NaN
    return value

ratings['rating'] = ratings['rating'].apply(replacewithNAN)
ratings

In [ ]:
# Construct the pivot table now
ratingsPivot = pd.pivot_table(ratings, values='rating', index='user_id', columns='anime_id')
ratingsPivot

In [ ]:
# Hamming Distance that omits NaN's for better comparisions
def hammingDist(user1, user2):
    matches = 0
    nans = 0
    for i in range(len(user1)):
        if np.isnan(user1[i]):
            nans += 1
            continue
        if np.isnan(user2[i]):
            nans += 1
            continue
        if user1[i] == user2[i]:
            matches += 1
    res = matches / (len(user1) - nans)
    return 1-res

In [ ]:
# Get the hamming distance between two users
from scipy.spatial import distance
def distanceFn(user1, user2):
    try:
        user1Ratings= ratingsPivot.transpose()[user1]
        user2Ratings= ratingsPivot.transpose()[user2]
        dist = distance.hamming(user1Ratings, user2Ratings)
    except:
        dist = np.NaN
    return dist

In [ ]:
# Find all the users and get the distance for each User
def KNearestNeighbors(activeUser, k=10):
    allUsers = pd.DataFrame(ratingsPivot.index)
    allUsers = allUsers[allUsers.user_id != activeUser]
    allUsers['distance'] = allUsers['user_id'].apply(lambda user: distanceFn(activeUser, user))
    topUsers = allUsers.sort_values(['distance'], ascending=[True])[:k]
    return topUsers

In [ ]:
def getBestSuggestions(activeuser, topUsers, animeCount=10):
    #     Get the nearest neighbours
    neighbourRatings = ratingsPivot[ratingsPivot.index.isin(topUsers['user_id'])]
    #     Get the average rating for all animes from the neighbors
    avgRatings = neighbourRatings.apply(np.nanmean).dropna()
    # Get all animes which dosn't have nan 
    animesWatched = ratingsPivot.transpose()[activeuser].dropna().index
    #     Delete animes you already watched to stop redundancies
    avgRatings = avgRatings[~avgRatings.index.isin(animesWatched)]
    #     Sort and get top 10 Animies
    topanime_ids = avgRatings.sort_values(ascending=False).index[:10]
    #     Return a dataframe to the user with top 10 recommendations
    recommendedAnimes = pd.DataFrame(columns = ['anime_id', 'name', 'genre'])
    for anime in topanime_ids:
        res = getAnimeDetails(anime)
        recommendedAnimes = recommendedAnimes.append({'anime_id' : res[0], 'name' : res[1], 'genre' : res[2]}, 
                    ignore_index = True)
    return recommendedAnimes

In [ ]:
def directorMethod(activeUser):
    topUsers = KNearestNeighbors(activeUser)
    return getBestSuggestions(activeUser, topUsers)

In [ ]:
activeUser = 73515
suggestions = directorMethod(activeUser)
suggestions

# Here is the Flow
1. Get the ratings dataset
2. If that's huge and spares drop that gives less information
3. Then construct the pivot table
4. Then calculate the distance b/w all users and pick the most nearest people
5. Then compute the averages for each anime with respect to all users
6. Then pick the top 5 or 10 with highest average
7. That's it now user the lookup table to get the names and genre of the shows

**                                     The END           **